Neste notebook será avaliado o quão boa foi a vetorização das Keywords. Para tal usamos dois conjuntos de teste composto por três colunas. No primeiro conjuto usamos as próprias keywords, as duas primeiras colunas contém keywords que foram extraídas de um mesmo artigo, já a terceira coluna tem um keyword aleatória. Espera-se que os vetores da primeira e segunda coluna estejam mais próximo entre si do que do vetor da keyword aleatória. O segundo conjunto de teste usa a mesma abordagem mas ao invés de usar keywords usa os autores dos artigos.

Importando bibliotecas

In [210]:
import pandas as pd
from gensim.models.doc2vec import Doc2Vec
from scipy import spatial

Lendo os arquivos

In [211]:
# Keyword Vectors
Keyword_vectors = pd.read_json("Elsevier_abstract - Keywords.json")
# Keyword Test Corpus
Test_Corpus_keywords = pd.read_csv("Test_Corpus_keywords.csv", index_col=0)
# Creator Vectors
Creator_vectors = pd.read_json("Elsevier_abstract - Creators.json")
# Creator Test Corpus
Test_Corpus_creator = pd.read_csv("Test_Corpus_creator.csv", index_col=0)
# Modelo Doc2Vec
doc2vec = Doc2Vec.load("AbstractKG_doc2vec_model")

In [212]:
# Vetores das Keywords
Creator_vectors.head()

,Abstracts,Creator,Doc2Vec
0,"[complexes, of, and, aminopyridine, with, the,...","gerrard, d.l.","[0.1216731519, -1.1021842957, 4.7513136864, 4...."
1,"[complexes, of, and, aminopyridine, with, the,...","allan, j.r.","[1.0570453405, 0.12217671420000001, 5.93484735..."
10,"[small, angle, neutron, scattering, has, been,...","peiffer, d.g.","[0.6661279202, -7.5089697838, 2.51254320139999..."
100,"[dijet, angular, distributions, from, the, fir...","liang, z.","[-1.2167836428, -6.4154033661, -7.2966299057, ..."
101,"[the, chemisorption, and, thermal, decompositi...","gland, j.l.","[3.8563585281, -5.672191143, 4.7051215172, 8.6..."


In [213]:
Test_Corpus_keywords.head()

,Keyword_1,Keyword_2,Random_keyword
0,xanes,weathering,environmental impact
1,iran,seismic,carbon capture and storage
2,transesterification,microwave,saft
3,transesterification,biodiesel,damage
4,microwave,biodiesel,steel


In [214]:
# Triplas de Keywords para avaliação
Test_Corpus_creator.head()

,Creator_1,Creator_2,Random_Creator
0,"allan, j.r.","paton, a.d.","petersen, j."
1,"allan, j.r.","turvey, k.","hovland, m."
2,"allan, j.r.","gerrard, d.l.","n’tsoukpoe, kokouvi edem"
3,"allan, j.r.","hoey, s.","schegelsky, v.a."
4,"paton, a.d.","turvey, k.","morley, c."


## Keywords

Unindo os corpus de teste com os vetores das keywords

In [215]:
# Criando arrays para os vetores
vector_1 = []
vector_2 = []
random_vector = []

# Para cada linha do corpus de teste identificar os vetores para cada keyword
for index, line in Test_Corpus_keywords.iterrows():
    vector_1.append(Keyword_vectors[Keyword_vectors["Keywords"]==line[0]]["Doc2Vec"].values[0])
    vector_2.append(Keyword_vectors[Keyword_vectors["Keywords"]==line[1]]["Doc2Vec"].values[0])
    random_vector.append(Keyword_vectors[Keyword_vectors["Keywords"]==line[2]]["Doc2Vec"].values[0])

In [216]:
# Incorporar os vetores no DataFrame "test_corpus"
Test_Corpus_keywords["vector_1"] = vector_1
Test_Corpus_keywords["vector_2"] = vector_2
Test_Corpus_keywords["random_vector"] = random_vector

In [217]:
# Incorporar as distâncias coseno entre os vetores
Test_Corpus_keywords["CosDist 1-2"] = Test_Corpus_keywords[["vector_1",
                                                            "vector_2"]].apply(lambda x: spatial.
                                                                               distance.
                                                                               cosine(*x),
                                                                               axis=1)
Test_Corpus_keywords["CosDist 1-Random"] = Test_Corpus_keywords[["vector_1",
                                                                 "random_vector"]].apply(lambda x: spatial.
                                                                                         distance.
                                                                                         cosine(*x),
                                                                                         axis=1)
Test_Corpus_keywords["CosDist 2-Random"] = Test_Corpus_keywords[["vector_2",
                                                                 "random_vector"]].apply(lambda x: spatial.
                                                                                         distance.
                                                                                         cosine(*x),
                                                                                         axis=1)

In [218]:
# O score é média entre as diferenças entre CostDist 1 - Random e CosDist 1 - 2, CostDist 2 - Random e CosDist 1 - 2
Test_Corpus_keywords["Score"] = ((Test_Corpus_keywords["CosDist 1-Random"] - 
                                  Test_Corpus_keywords["CosDist 1-2"]) +
                                 (Test_Corpus_keywords["CosDist 2-Random"] - 
                                  Test_Corpus_keywords["CosDist 1-2"])) / 2

In [219]:
# O Score_bool é True quando CosDist 1-2 é maior que as distancias para o vetor aleatório. 
# A métrica é dado pelo percentual de True no total
Test_Corpus_keywords["Score_bool"] = ((Test_Corpus_keywords["CosDist 1-Random"] >
                                       Test_Corpus_keywords["CosDist 1-2"]) &
                                      (Test_Corpus_keywords["CosDist 2-Random"] > 
                                       Test_Corpus_keywords["CosDist 1-2"]))

Calculando o score para as keywords

In [220]:
keyword_score = Test_Corpus_keywords["Score"].mean()
keyword_score_bool = (Test_Corpus_keywords["Score_bool"].sum() /
                      len(Test_Corpus_keywords["Score_bool"]))

print ("O Keyword Score foi: ", keyword_score)
print ("O Keyword Score_bool foi: ", keyword_score_bool)

O Keyword Score foi:  0.6423788514944921
O Keyword Score_bool foi:  0.8041965528316026


## Autores

Unindo os corpus de teste com os vetores dos autores

In [221]:
# Criando arrays para os vetores
vector_1 = []
vector_2 = []
random_vector = []

# Para cada linha do corpus de teste identificar os vetores para cada autor

for index, line in Test_Corpus_creator.iterrows():
    try:        
        vector_1.append(Creator_vectors[Creator_vectors["Creator"]==line[0]]["Doc2Vec"].values[0])
    except:
        vector_1.append('branco')
    
    try:
        vector_2.append(Creator_vectors[Creator_vectors["Creator"]==line[1]]["Doc2Vec"].values[0])
    except:
        vector_2.append('branco')
    
    try:
        random_vector.append(Creator_vectors[Creator_vectors["Creator"]==line[2]]["Doc2Vec"].values[0])
    except:
        random_vector.append('branco')

In [222]:
# Incorporar os vetores no DataFrame "Test_Corpus_creator"
Test_Corpus_creator["vector_1"] = vector_1
Test_Corpus_creator["vector_2"] = vector_2
Test_Corpus_creator["random_vector"] = random_vector

In [223]:
# Retirando os vetores que retornaram em branco
Test_Corpus_creator = Test_Corpus_creator[(Test_Corpus_creator["vector_1"]!='branco') &
                                          (Test_Corpus_creator["vector_2"]!='branco') & 
                                          (Test_Corpus_creator["random_vector"]!='branco')]

In [224]:
# Incorporar as distâncias coseno entre os vetores
Test_Corpus_creator["CosDist 1-2"] = Test_Corpus_creator[["vector_1",
                                                          "vector_2"]].apply(lambda x: spatial.
                                                                             distance.
                                                                             cosine(*x),
                                                                             axis=1)
Test_Corpus_creator["CosDist 1-Random"] = Test_Corpus_creator[["vector_1",
                                                               "random_vector"]].apply(lambda x: spatial.
                                                                                       distance.
                                                                                       cosine(*x),
                                                                                       axis=1)
Test_Corpus_creator["CosDist 2-Random"] = Test_Corpus_creator[["vector_2",
                                                               "random_vector"]].apply(lambda x: spatial.
                                                                                       distance.
                                                                                       cosine(*x),
                                                                                       axis=1)

In [225]:
# O score é média entre as diferenças entre CostDist 1 - Random e CosDist 1 - 2, CostDist 2 - Random e CosDist 1 - 2
Test_Corpus_creator["Score"] = ((Test_Corpus_creator["CosDist 1-Random"] - 
                                 Test_Corpus_creator["CosDist 1-2"]) +
                                (Test_Corpus_creator["CosDist 2-Random"] - 
                                 Test_Corpus_creator["CosDist 1-2"])) / 2

In [226]:
# O Score_bool é True quando CosDist 1-2 é maior que as distancias para o vetor aleatório. 
# A métrica é dado pelo percentual de True no total
Test_Corpus_creator["Score_bool"] = ((Test_Corpus_creator["CosDist 1-Random"] >
                                      Test_Corpus_creator["CosDist 1-2"]) &
                                     (Test_Corpus_creator["CosDist 2-Random"] > 
                                      Test_Corpus_creator["CosDist 1-2"]))

Calculando o score para os autores

In [227]:
creator_score = Test_Corpus_creator["Score"].mean()
creator_score_bool = (Test_Corpus_creator["Score_bool"].sum() /
                      len(Test_Corpus_creator["Score_bool"]))

print ("O Creator Score foi: ", creator_score)
print ("O Creator Score_bool foi: ", creator_score_bool)

O Creator Score foi:  0.8353482928502103
O Creator Score_bool foi:  0.96982915303526


## Score Global

Consideramos o score global como a média dos scores das keywords e dos autores

In [228]:
print ("O Global Score foi: ", (creator_score + keyword_score) / 2)
print ("O Global Score_bool foi: ", (creator_score_bool + keyword_score_bool) / 2)

O Global Score foi:  0.7388635721723512
O Global Score_bool foi:  0.8870128529334314
